In [1]:
import numpy as np
import argparse
import c3.libraries.fidelities as fidelities
from utils import *
import c3.libraries.algorithms as algorithms
import c3.utils.qt_utils as qt_utils
import c3.libraries.constants as constants
from SingleQubitExperiment import SingleQubitExperiment

2021-10-05 16:57:23.341965: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-05 16:57:23.342004: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("output", help="Output directory")
args = parser.parse_args()
output_dir = args.output
#output_dir = './output'

In [3]:
def spectral_peaks_goal(propagators: dict, instructions: dict, index, dims,
                        qubit: chip.Qubit, generator: Generator, drive: chip.Drive,
                        n_eval=-1, num_peaks=2) -> tf.Tensor:
    # find energies of the qubit
    energies = qubit.get_Hamiltonian().numpy().diagonal().real
    vec1 = np.array([energies[3] - energies[2], energies[1] - energies[0]]) / (2 * np.pi)

    infids = []
    for gate, propagator in propagators.items():
        # create signal and find peaks
        instr = instructions[gate]
        signal = generator.generate_signals(instr)[drive.name]
        peakFrequencies, peakValues = findFrequencyPeaks(signal["ts"].numpy(), signal["values"].numpy(), num_peaks)
        peakFrequencies = tf.convert_to_tensor(peakFrequencies)
        peakValues = tf.convert_to_tensor(peakValues)

        # add frequency offsets to infidelity
        #infid = np.linalg.norm(vec1 - peakFrequencies)
        infid1 = tf.abs(vec1[0] - peakFrequencies[0]) / peakFrequencies[0]
        infid1 += tf.abs(vec1[1] - peakFrequencies[1]) / peakFrequencies[1]

        # add peak height difference to infidelity
        if peakFrequencies[1] > peakFrequencies[0]:
            a = peakValues[1]
            b = np.sqrt(2) * peakValues[0]
        else:
            a = peakValues[0]
            b = np.sqrt(2) * peakValues[1]
        infid2 = tf.abs(a - b) / tf.maximum(a, b)
        infid = infid1 * (1 + infid2)
        infids.append(infid)
    result = tf.reduce_mean(infids)
    print("type: ", type(result))
    return result


def printInformation(exper: SingleQubitExperiment, name: str) -> None:
    signal = exper.generateSignal()
    peakFrequencies, peakValues = findFrequencyPeaks(signal["ts"].numpy(), signal["values"].numpy(), 4)
    print("peaks: ", np.sort(peakFrequencies))
    exper.saveSignal("signal_" + name)
    exper.plotSignal("signal_" + name)
    exper.saveTimeEvolution("populations_" + name)
    exper.plotTimeEvolution("populations_" + name)
    exper.savePropagator("propagator_" + name)
    exper.plotPropagator("propagator_" + name)

In [4]:
freq = 5e9
anharm = 300e6
t_final = 15e-9
sigma = t_final / 15
sigma2 = sigma / 2
relative_amp = 2
carrier_freq = 4.7e9
optimisable_params = {
    "pwc": [
        "amp",
        "xy_angle", "freq_offset",
        "delta",
        "t_final", #"t_bin_start", "t_bin_end",
        "inphase"
    ],
    "carrier": [
        "freq",
        "framechange"
    ]
}

In [6]:
envelope = createPWCDoubleGaussianPulse(t_final, sigma, sigma2, relative_amp, 30)
exper = SingleQubitExperiment(directory=f"{output_dir}/")
exper.prepare(t_final, freq, anharm, carrier_freq, envelope,
              #ideal_gate=qt_utils.np_kron_n([constants.Id, constants.y90p]),
              #ideal_gate=constants.GATES["cx"],
              #ideal_gate=np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]], dtype=np.complex128),
              ideal_gate=qt_utils.np_kron_n([constants.Id, constants.y90p]),
              useDrag=True)

# print information
energies = exper.getEnergies()
print("energies: ", [(energies[i + 1] - energies[i]) / (2 * np.pi) for i in range(len(energies) - 1)])
printInformation(exper, "before")

# first optimisation
callback = lambda infid: print(infid)
params_after = exper.optimise(
    optimisable_params,
    algorithm=algorithms.lbfgs,
    algorithm_options={
        #"stop_at_convergence": 15,
        #"spread": 0.2,
        #"popsize": 25,
        "maxfun": 200,
    },
    fidelity_function=fidelities.unitary_infid_set_with_penalty,
    fidelity_params={
        "active_levels": 4,
        #"qubit": exper.getQubit(),
        #"generator": exper.getExperiment().pmap.generator,
        #"drive": getDrive(exper.getExperiment().pmap.model, exper.getQubit())
        "penalty_factor": 1
    },
    callback=callback,
)
printInformation(exper, "after")
print(params_after)

# second experiment
'''
    envelope2 = createPWCCustomPulse(t_final, optimisedParams['lower-X[0]-d1-custom-sigma'], optimisedParams['lower-X[0]-d1-custom-cosine_frequency'], 30)
    exper2 = SingleQubitExperiment(directory=f"{output_dir}/", file_suffix=f"{i}")
    carrier_freq2 = optimisedParams['lower-X[0]-d1-carrier-freq']
    exper2.prepare(t_final, freq, anharm, carrier_freq2, envelope2, useDrag=False)

    # second optimisation
    callback = lambda infid: print("infidelity: ", infid)
    params_after = exper.optimise(
        optimisable_params,
        algorithm=algorithms.lbfgs,
        algorithm_options={
            #"stop_at_convergence": 15,
            #"spread": 0.15,
            #"popsize": 20,
            "maxfun": 100
        },
        fidelity_function=fidelities.unitary_infid_set,
        fidelity_params={
            "active_levels": 4,
        },
        callback=callback
    )
    printInformation(exper, "after")
    print(params_after)
    '''

NameError: name 'output_dir' is not defined